# Feed Forward Neural Network Example

Code taken from:

https://pyimagesearch.com/2021/05/06/backpropagation-from-scratch-with-python/

Backpropagation from scratch with Python
by Adrian Rosebrock on May 6, 2021

**Prerequisites:**

In [ ]:
!pip install -U numpy

In [1]:
import numpy as np

class NeuralNetwork:
    def __init__(self, layers, alpha=0.1):
        # initialize the list of weights matrices, then store the
        # network architecture and learning rate
        self.W = []
        self.layers = layers
        self.alpha = alpha

        # start looping from the index of the first layer but
        # stop before we reach the last two layers
        for i in np.arange(0, len(layers) - 2):
            # randomly initialize a weight matrix connecting the
            # number of nodes in each respective layer together,
            # adding an extra node for the bias
            w = np.random.randn(layers[i] + 1, layers[i + 1] + 1)
            self.W.append(w / np.sqrt(layers[i]))

        # the last two layers are a special case where the input
        # connections need a bias term but the output does not
        w = np.random.randn(layers[-2] + 1, layers[-1])
        self.W.append(w / np.sqrt(layers[-2]))

    def __repr__(self):
        # construct and return a string that represents the network
        # architecture
        return "NeuralNetwork: {}".format(
            "-".join(str(l) for l in self.layers))

    def sigmoid(self, x):
        # compute and return the sigmoid activation value for a
        # given input value
        return 1.0 / (1 + np.exp(-x))

    def sigmoid_deriv(self, x):
        # compute the derivative of the sigmoid function ASSUMING
        # that x has already been passed through the 'sigmoid'
        # function
        return x * (1 - x)

    def fit(self, X, y, epochs=1000, displayUpdate=100):
        # insert a column of 1's as the last entry in the feature
        # matrix -- this little trick allows us to treat the bias
        # as a trainable parameter within the weight matrix
        X = np.c_[X, np.ones((X.shape[0]))]

        # loop over the desired number of epochs
        for epoch in np.arange(0, epochs):
            # loop over each individual data point and train
            # our network on it
            for (x, target) in zip(X, y):
                self.fit_partial(x, target)

            # check to see if we should display a training update
            if epoch == 0 or (epoch + 1) % displayUpdate == 0:
                loss = self.calculate_loss(X, y)
                print("[INFO] epoch={}, loss={:.7f}".format(
                    epoch + 1, loss))

    def fit_partial(self, x, y):
        # construct our list of output activations for each layer
        # as our data point flows through the network; the first
        # activation is a special case -- it's just the input
        # feature vector itself
        A = [np.atleast_2d(x)]

        # FEEDFORWARD:
        # loop over the layers in the network
        for layer in np.arange(0, len(self.W)):
            # feedforward the activation at the current layer by
            # taking the dot product between the activation and
            # the weight matrix -- this is called the "net input"
            # to the current layer
            net = A[layer].dot(self.W[layer])

            # computing the "net output" is simply applying our
            # nonlinear activation function to the net input
            out = self.sigmoid(net)

            # once we have the net output, add it to our list of
            # activations
            A.append(out)

        # BACKPROPAGATION
        # the first phase of backpropagation is to compute the
        # difference between our *prediction* (the final output
        # activation in the activations list) and the true target
        # value
        error = A[-1] - y

        # from here, we need to apply the chain rule and build our
        # list of deltas 'D'; the first entry in the deltas is
        # simply the error of the output layer times the derivative
        # of our activation function for the output value
        D = [error * self.sigmoid_deriv(A[-1])]

        # once you understand the chain rule it becomes super easy
        # to implement with a 'for' loop -- simply loop over the
        # layers in reverse order (ignoring the last two since we
        # already have taken them into account)
        for layer in np.arange(len(A) - 2, 0, -1):
            # the delta for the current layer is equal to the delta
            # of the *previous layer* dotted with the weight matrix
            # of the current layer, followed by multiplying the delta
            # by the derivative of the nonlinear activation function
            # for the activations of the current layer
            delta = D[-1].dot(self.W[layer].T)
            delta = delta * self.sigmoid_deriv(A[layer])
            D.append(delta)

        # since we looped over our layers in reverse order we need to
        # reverse the deltas
        D = D[::-1]

        # WEIGHT UPDATE PHASE
        # loop over the layers
        for layer in np.arange(0, len(self.W)):
            # update our weights by taking the dot product of the layer
            # activations with their respective deltas, then multiplying
            # this value by some small learning rate and adding to our
            # weight matrix -- this is where the actual "learning" takes
            # place
            self.W[layer] += -self.alpha * A[layer].T.dot(D[layer])

    def predict(self, X, addBias=True):
        # initialize the output prediction as the input features -- this
        # value will be (forward) propagated through the network to
        # obtain the final prediction
        p = np.atleast_2d(X)

        # check to see if the bias column should be added
        if addBias:
            # insert a column of 1's as the last entry in the feature
            # matrix (bias)
            p = np.c_[p, np.ones((p.shape[0]))]

        # loop over our layers in the network
        for layer in np.arange(0, len(self.W)):
            # computing the output prediction is as simple as taking
            # the dot product between the current activation value 'p'
            # and the weight matrix associated with the current layer,
            # then passing this value through a nonlinear activation
            # function
            p = self.sigmoid(np.dot(p, self.W[layer]))

        # return the predicted value
        return p

    def calculate_loss(self, X, targets):
        # make predictions for the input data points then compute
        # the loss
        targets = np.atleast_2d(targets)
        predictions = self.predict(X, addBias=False)
        loss = 0.5 * np.sum((predictions - targets) ** 2)

        # return the loss
        return loss

In [7]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

In [10]:
nn = NeuralNetwork([2, 2, 1], alpha=0.5)
nn.fit(X, y, epochs=20000)

[INFO] epoch=1, loss=0.5379580
[INFO] epoch=100, loss=0.5001733
[INFO] epoch=200, loss=0.4998304
[INFO] epoch=300, loss=0.4996054
[INFO] epoch=400, loss=0.4993540
[INFO] epoch=500, loss=0.4988470
[INFO] epoch=600, loss=0.4975845
[INFO] epoch=700, loss=0.4939464
[INFO] epoch=800, loss=0.4819670
[INFO] epoch=900, loss=0.4468838
[INFO] epoch=1000, loss=0.3891372
[INFO] epoch=1100, loss=0.3323508
[INFO] epoch=1200, loss=0.2748918
[INFO] epoch=1300, loss=0.2228820
[INFO] epoch=1400, loss=0.1883357
[INFO] epoch=1500, loss=0.1676274
[INFO] epoch=1600, loss=0.1538164
[INFO] epoch=1700, loss=0.1392713
[INFO] epoch=1800, loss=0.0756454
[INFO] epoch=1900, loss=0.0267888
[INFO] epoch=2000, loss=0.0169463
[INFO] epoch=2100, loss=0.0126791
[INFO] epoch=2200, loss=0.0102043
[INFO] epoch=2300, loss=0.0085621
[INFO] epoch=2400, loss=0.0073835
[INFO] epoch=2500, loss=0.0064928
[INFO] epoch=2600, loss=0.0057942
[INFO] epoch=2700, loss=0.0052308
[INFO] epoch=2800, loss=0.0047664
[INFO] epoch=2900, loss=0.

In [11]:
for (x, target) in zip(X, y):
	# make a prediction on the data point and display the result
	# to our console
	pred = nn.predict(x)[0][0]
	step = 1 if pred > 0.5 else 0
	print("[INFO] data={}, ground-truth={}, pred={:.4f}, step={}".format(
		x, target[0], pred, step))

[INFO] data=[0 0], ground-truth=0, pred=0.0087, step=0
[INFO] data=[0 1], ground-truth=1, pred=0.9866, step=1
[INFO] data=[1 0], ground-truth=1, pred=0.9893, step=1
[INFO] data=[1 1], ground-truth=0, pred=0.0129, step=0


Change the network architecture:

In [12]:
nn = NeuralNetwork([2, 1], alpha=0.5)
nn.fit(X, y, epochs=20000)

[INFO] epoch=1, loss=0.5274173
[INFO] epoch=100, loss=0.5010323
[INFO] epoch=200, loss=0.5007965
[INFO] epoch=300, loss=0.5007934
[INFO] epoch=400, loss=0.5007937
[INFO] epoch=500, loss=0.5007938
[INFO] epoch=600, loss=0.5007938
[INFO] epoch=700, loss=0.5007938
[INFO] epoch=800, loss=0.5007938
[INFO] epoch=900, loss=0.5007938
[INFO] epoch=1000, loss=0.5007938
[INFO] epoch=1100, loss=0.5007938
[INFO] epoch=1200, loss=0.5007938
[INFO] epoch=1300, loss=0.5007938
[INFO] epoch=1400, loss=0.5007938
[INFO] epoch=1500, loss=0.5007938
[INFO] epoch=1600, loss=0.5007938
[INFO] epoch=1700, loss=0.5007938
[INFO] epoch=1800, loss=0.5007938
[INFO] epoch=1900, loss=0.5007938
[INFO] epoch=2000, loss=0.5007938
[INFO] epoch=2100, loss=0.5007938
[INFO] epoch=2200, loss=0.5007938
[INFO] epoch=2300, loss=0.5007938
[INFO] epoch=2400, loss=0.5007938
[INFO] epoch=2500, loss=0.5007938
[INFO] epoch=2600, loss=0.5007938
[INFO] epoch=2700, loss=0.5007938
[INFO] epoch=2800, loss=0.5007938
[INFO] epoch=2900, loss=0.

In [13]:
for (x, target) in zip(X, y):
	# make a prediction on the data point and display the result
	# to our console
	pred = nn.predict(x)[0][0]
	step = 1 if pred > 0.5 else 0
	print("[INFO] data={}, ground-truth={}, pred={:.4f}, step={}".format(
		x, target[0], pred, step))

[INFO] data=[0 0], ground-truth=0, pred=0.5161, step=1
[INFO] data=[0 1], ground-truth=1, pred=0.5000, step=0
[INFO] data=[1 0], ground-truth=1, pred=0.4839, step=0
[INFO] data=[1 1], ground-truth=0, pred=0.4678, step=0
